In [6]:
import base64
import io
import openai
import os
import pandas as pd
import requests
import textwrap
from PIL import Image, ImageDraw, ImageFont

# OpenAI - API

In [ ]:
# load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_timeout = 10

In [ ]:
# list engines
engines = openai.Engine.list()

# print the first engine's id
print([i.id for i in engines.data])

In [ ]:
# TODO: list roles
role = "Your task is to write prompts that will be used in text to image models for generating memes. \
        Your response can have multiple prompts in the following format for each: a long prompt that describes a funny and agitative situtation starting with 'Prompt:', \
        then the caption of the corresponding meme that follows the prompt starting with 'Caption:'."
user = "Please try to come up with imaginative, very funny and novel memes, that describes specific situations. \
        Memes should be sarcastic and didactic, and should make laugh people while make them thinking about the society they live in. \
        Would be great to have the main themes about critism of capitalism, critism of commodity fetishism, alienation, critisim of while collar people and corporate culture, or class inequality. \
        Ideologically, these memes should be positioned in economic-left."

In [ ]:
#df_prompt = pd.DataFrame(columns=["prompt", "caption", "seed"])
df_prompt = pd.read_csv("df_prompt.csv")
print(df_prompt.shape)

In [ ]:
def gpt_request():
    # ask the prompt
    response = ""
    try:
        response = openai.ChatCompletion.create(
            model= "gpt-3.5-turbo", # "gpt-4", "gpt-3.5-turbo",
            temperature=1,
            messages=[
                {"role": "system", "content": role},
                {"role": "user", "content": user},
            ]
        )
    except:
        pass
    # return result
    result = ''
    try:
        for choice in response.choices:
            result += choice.message.content
    except: 
        pass
    return result

In [ ]:
#prompt = gpt_request()
#prompt

# Stability - API

In [ ]:
# constants
url = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": os.getenv("STABILITY_AI_API_KEY"),
}

In [ ]:
# TODO: set meme style
style = " The meme should be simple, featuring bold, caricatured characters with vivid colours, via very funny and imaginative imagery."

In [ ]:
def stability_prompt_body(prompt):
    return {
        "steps": 40,
        "width": 1024,
        "height": 1024,
        "seed": 0,
        "cfg_scale": 5,
        "samples": 1,
        "text_prompts": [
            {
            "text": "Draw a meme which depicts: " + prompt + style,
            "weight": 1
            },
            {
                "text": "blurry, text, text box cloud",
                "weight": -1
            }
        ],
    }

# Image & Caption

In [ ]:
def split_sentence(sentence):
    words = sentence.split()
    middle_index = len(words) // 2
    words.insert(middle_index, '\n')
    return ' '.join(words)

In [ ]:
def add_caption_and_save(image, seed, caption):
    # create a new image with a white background, slightly taller to accommodate the caption
    width, height = image.size
    new_height = height + 100  # Adjust this value as needed
    new_image = Image.new("RGB", (width, new_height), "white")

    # paste the original image onto the new image
    new_image.paste(image, (0, 0))

    # create a drawing context on the new image
    draw = ImageDraw.Draw(new_image)

    # calculate text size and position based on font size and caption
    font = ImageFont.truetype("impact.ttf", 35)
    caption = caption.upper()
    caption_wrap = textwrap.wrap(caption, width=60)
    current_h, pad = height, 10
    for line in caption_wrap:
        w, h = draw.textsize(line, font=font)
        draw.text(((width - w) / 2, current_h), line, fill="black", font=font)
        current_h += h + pad

    # save the new image with the caption
    new_image.save(f'./memes/meme_{str(seed)}.png')

    # close the original image
    new_image.close()

In [ ]:
"""
image = Image.open("memes/meme_2486965645.png")
seed = "2486965645"
caption = "Commodity Fetishism in the Modern Workplace: When productivity becomes a circus act and creativity is traded for chaos."
add_caption_and_save(image, seed, caption)
"""

# Generate Memes

In [ ]:
# generate memes
for single_response in range(1):
    single_response = gpt_request()
    for i in single_response.split("Prompt:")[1:]:
        # seperate caption from prompt
        prompt = i.split("Caption:")[0].strip()
        caption = i.split("Caption:")[1].strip().replace('"', '')
        print("\nPrompt:", prompt)
        print("Caption:", caption)

        # forward prompt to stability ai
        response = requests.post(
            url,
            headers=headers,
            json=stability_prompt_body(prompt),
        )
        if response.status_code != 200:
            raise Exception("Non-200 response: " + str(response.text))

        # get image and add caption into it
        data = response.json()
        seed = data["artifacts"][0]["seed"]
        image = Image.open(io.BytesIO(base64.decodebytes(bytes(data["artifacts"][0]["base64"], "utf-8"))))
        add_caption_and_save(image, seed, caption)

        # add into the dataframe
        new_row = {'prompt': prompt, 'caption': caption, 'seed':seed}
        df_prompt = pd.concat([df_prompt, pd.DataFrame([new_row])], ignore_index=True)
        
# save all prompts
df_prompt.to_csv("df_prompt.csv", index=False)

In [ ]:
df_prompt